In [1]:
import os
from getpass import getpass
os.environ['OPENAI_API_KEY'] = getpass('OpenAI API Key:')

In [8]:
## Init main variables

root_path = "/Users/ryanlindbeck/Development/resai/app/backend/notebooks"
doc_path = f"{root_path}/documents"
db_path = f"{root_path}/vector_deeplake.db"
documents_dir = "fin570"

In [9]:
question = """
germany
"""

In [10]:
## Load PDF document into DeepLake with Langchain.

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import DeepLake
from langchain.document_loaders import PyPDFDirectoryLoader

embedding_function = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

loader = PyPDFDirectoryLoader(f"{doc_path}/{documents_dir}")

documents = loader.load_and_split()
texts = [d.page_content for d in documents]
metadatas = [d.metadata for d in documents]

store = DeepLake(dataset_path=db_path, embedding_function=embedding_function)

store.add_texts(texts, metadatas=metadatas)

# Refs:
# https://docs.activeloop.ai/tutorials/vector-store/deep-lake-vector-store-in-langchain

Dataset(path='/Users/ryanlindbeck/Development/resai/app/backend/notebooks/vector_deeplake.db', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape      dtype  compression
  -------    -------    -------    -------  ------- 
 embedding  embedding  (838, 384)  float32   None   
    id        text      (838, 1)     str     None   
 metadata     json      (838, 1)     str     None   
   text       text      (838, 1)     str     None   


['000706f8-1bad-11ee-a866-7ed7efac1b81',
 '00070784-1bad-11ee-a866-7ed7efac1b81',
 '000707a2-1bad-11ee-a866-7ed7efac1b81',
 '000707c0-1bad-11ee-a866-7ed7efac1b81',
 '000707d4-1bad-11ee-a866-7ed7efac1b81',
 '000707e8-1bad-11ee-a866-7ed7efac1b81',
 '000707fc-1bad-11ee-a866-7ed7efac1b81',
 '00070824-1bad-11ee-a866-7ed7efac1b81',
 '00070838-1bad-11ee-a866-7ed7efac1b81',
 '0007084c-1bad-11ee-a866-7ed7efac1b81',
 '00070860-1bad-11ee-a866-7ed7efac1b81',
 '00070874-1bad-11ee-a866-7ed7efac1b81',
 '0007087e-1bad-11ee-a866-7ed7efac1b81',
 '00070892-1bad-11ee-a866-7ed7efac1b81',
 '000708a6-1bad-11ee-a866-7ed7efac1b81',
 '000708b0-1bad-11ee-a866-7ed7efac1b81',
 '000708c4-1bad-11ee-a866-7ed7efac1b81',
 '000708d8-1bad-11ee-a866-7ed7efac1b81',
 '000708e2-1bad-11ee-a866-7ed7efac1b81',
 '000708f6-1bad-11ee-a866-7ed7efac1b81',
 '0007090a-1bad-11ee-a866-7ed7efac1b81',
 '00070914-1bad-11ee-a866-7ed7efac1b81',
 '00070928-1bad-11ee-a866-7ed7efac1b81',
 '00070932-1bad-11ee-a866-7ed7efac1b81',
 '00070946-1bad-

In [11]:
search_result_documents = store.similarity_search(question, k=5)

print(f"Total Results: {len(search_result_documents)}")

# sort search results by page number
sorted_results = sorted(search_result_documents, key=lambda doc: int(doc.metadata['page']))

# The materials 
doc_prompts: list[str] = []

page_numbers: list[str] = []

for doc in sorted_results:
    page_content = doc.page_content
    source = ''
    page = ''
    
    if 'page' in doc.metadata:
        page = doc.metadata["page"]
        page_numbers.append(page)

    if 'source' in doc.metadata:
        source = doc.metadata["source"]

    doc_prompts.append(
f"""
Material Content:
{page_content}

Material Metadata:
Source: {source}
Page: {page}
"""
    )
    

print(f"Page Numbers: {page_numbers}")
print("Doc Prompts")
print(doc_prompts)

Total Results: 5
Page Numbers: [1, 21, 25, 57, 104]
Doc Prompts
["\nMaterial Content:\nCorporate Finance I: Measuring and Promoting Value Creation Professors Stefan Zeume & Heitor Almeida Module 1: The Objective and Language of Corporate Finance    Lesson 1-1.0: Module Objectives and Overview  \n This first module is about the goals and the language of corporate finance. Haters from Brazil, I'm from Germany. We thought we would start the course with a football or soccer ball depending on where you come from. Unfortunately, though the language of corporate finance is not Portuguese or German like hater and I learned when we were kids.\n\nMaterial Metadata:\nSource: /Users/ryanlindbeck/Development/resai/app/backend/notebooks/documents/fin570/module1.pdf\nPage: 1\n", "\nMaterial Content:\nCorporate Finance Ⅱ:  Financing Investments and Managing Risk Professor Heitor Almeida and Stefan Zeume \n 22  Okay, so let me give you a specific example that I'm pulling out from this table and you can

In [ ]:
from langchain import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI

llm = OpenAI(temperature=0.0)
# llm = ChatOpenAI(temperature=0, model="gpt-4")

documents_text = "\n".join(doc_prompts)

print("Question: ")
print(question)
print("")
print("Materials:")
print(documents_text)
print("")

template = """
{template_heading}

Question: 
{question}

Possible Answers:
{possible_answers}

Materials:
{documents_text}
"""

# prompt = PromptTemplate(
#     input_variables=["template_heading", "question", "possible_answers", "documents_text"],
#     template=template,
# )

# chain = LLMChain(llm=llm, prompt=prompt)

# print("")
# print("Response:")
# print(chain.run({
#     'template_heading': template_heading,
#     'question': question,
#     'possible_answers': possible_answers,
#     'documents_text': documents_text
# }))

In [14]:
from deeplake.core.vectorstore.deeplake_vectorstore import DeepLakeVectorStore

vector_store = DeepLakeVectorStore(path = db_path)

def embedding_function_hf(texts, model="all-MiniLM-L6-v2"):
   
   if isinstance(texts, str):
       texts = [texts]

   texts = [t.replace("\n", " ") for t in texts]
   return [data['embedding']for data in openai.Embedding.create(input = texts, model=model)['data']]

# todo:

# list documents
# search documents
# Update collection.
# Add new documents to collection.
# Remove documents from collection.
# Delete collection.

prompt = "What are the first programs he tried writing?"

search_results = vector_store.search(embedding_data=prompt, embedding_function=embedding_function_hf)

search_results['text'][0]

Deep Lake Dataset in /Users/ryanlindbeck/Development/resai/app/backend/notebooks/vector_deeplake.db already exists, loading from the storage


TypeError: 'HuggingFaceEmbeddings' object is not callable